In [ ]:
import os  # operating system functions like renaming files and directories
import shutil  # recursive file and directory operations
import glob  # pattern matching for paths
import pandas as pd  # data mangling and transforming
import bandicoot as bc  # MIT toolkit for creating bandicoot indicators
import argparse  # entering flags from the cmd line
import gnuper as gn  # the package in question
from pyspark.sql import SparkSession  # using spark context for big data files
from pyspark.sql.functions import col  # needed for function over each column

In [ ]:
mp_flag = True
bc_flag = True
verbose = True
clean_up = False
raw_data_path = 'sample_data/'

In [ ]:
# define attributes for this session
att = gn.Attributes(mp_flag=mp_flag, bc_flag=bc_flag, verbose=verbose, clean_up=clean_up,
                    raw_data_path=raw_data_path,
                    cap_coords=(15.500654, 32.559899),  # capital gps (lat, lon)
                    weekend_days=[5, 6])